In [10]:
import pandas as pd
import numpy as np

import glob, re, os

In [24]:
subject = 860

In [25]:
event_fns = np.sort(glob.glob('/Users/67981492/Desktop/loki_1/data/BIDS/sub-0{}/ses-*/beh/*task-lokicat*.tsv'.format(subject)))
onset_fns = np.sort(glob.glob('/Users/67981492/Desktop/loki_1/data/BIDS/sub-0{}/ses-*/func/*task-lokicat*events.tsv'.format(subject)))

event_fns.sort()
onset_fns.sort()

In [26]:
try: 
    assert len(event_fns) == len(onset_fns), 'Events & behavior files are of different lengths! Check them.'
except: 
    event_ids = [event[65:94] for event in event_fns]
    onset_ids = [onset[66:95] for onset in onset_fns]


    onset_greater = len(onset_ids) > len(event_ids)

    if onset_greater: 
        missing_fn = list(set(onset_ids).difference(event_ids))
        assert len(missing_fn) == 1, 'check length of missing files'
        onset_ids.remove(missing_fn[0])
    else: 
        missing_fn = list(set(event_ids).difference(onset_ids))
        assert len(missing_fn) == 1, 'check length of missing files'
        event_ids.remove(missing_fn[0])
    
    
    event_fns = []
    for id in event_ids:
        event_fns.append(glob.glob('/Users/67981492/Desktop/loki_1/data/BIDS/sub-0790/ses-*/beh/*{}*.tsv'.format(id))[0])
    
    onset_fns = []
    for id in onset_ids:
        onset_fns.append(glob.glob('/Users/67981492/Desktop/loki_1/data/BIDS/sub-0790/ses-*/func/*{}*.tsv'.format(id))[0])


    assert len(event_fns) == len(onset_fns), 'Events & behavior files are of different lengths! Check them.'

In [27]:
for event, onset in zip(event_fns, onset_fns):

    events_subject = event.partition('sub-0')[2][:3]
    events_session_n = event.partition('ses-')[2][:2]
    events_run_n = event.partition('run')[2][:2]
    
    onset_subject = onset.partition('sub-0')[2][:3]
    onset_session_n = onset.partition('ses-')[2][:2]
    onset_run_n = onset.partition('run')[2][:2]
    

    print(events_subject, events_session_n, events_run_n, onset_subject, onset_session_n, onset_run_n)    

    assert(events_subject, events_session_n, events_run_n) == (onset_subject, onset_session_n, onset_run_n), 'check matching of files'
    
    events_path = ('/Users/67981492/Desktop/loki_1/data/BOLD/sub-{}/fmriprep/sub-{}/ses-{}/func/').format(onset_subject, onset_subject, onset_session_n) 
    
    events_df = pd.read_csv(event, sep=r'\,|\t',engine='python').rename(columns=lambda x: x.strip())
    
    # load onsets and items
    try:
      onsets_df = pd.read_csv(onset, delim_whitespace=True).rename(columns=lambda x: x.strip())
      onsets_df.rt
    except:
      onsets_df = pd.read_csv(onset, sep=r'\,|\t',engine='python').rename(columns=lambda x: x.strip())

    
    event_design_df = pd.DataFrame()


    event_design_df['reward_code'] = events_df['reward_code'].copy() 
    event_design_df['subj_id'] = events_df['subj_id'].copy() 
    event_design_df['condition'] = events_df['condition'].copy() 
    event_design_df['trial'] = events_df['trial'].copy() 
    event_design_df['epoch_trial'] = events_df['epoch_trial'].copy() 
    event_design_df['epoch_number'] = events_df['epoch_number'].copy() 
    event_design_df['button_press'] = events_df['abs_response_time'].copy() 
    
    
    event_design_df['LR_solution'] = events_df['p_LR_solution']
    event_design_df['LR'] = events_df['LR_choice'].copy() 
    event_design_df['reward'] = events_df['reward'].copy()
    event_design_df['p_accuracy'] = events_df['p_accuracy'].copy()
    event_design_df['rt'] = events_df['rt'].copy() 


    event_design_df['trial'] = events_df['trial'].copy() 
    event_design_df['trial_end'] = events_df['stim_offset']
    event_design_df['iti_length'] = events_df['iti']
    event_design_df['iti_end'] = events_df['iti'] + events_df['stim_offset']



        
    try: 
        event_design_df['stim_onset'] = onsets_df['stim_onset']

    except: 
        event_design_df['stim_onset'] = onsets_df['onset']
    


    event_design_df['session_n'] = int(events_session_n)
    event_design_df['run_n'] = int(events_run_n)
    
    print('Number of nonresponse trials: ', 
          event_design_df['rt'].isnull().values.sum())
    
    event_design_df.dropna(inplace=True) # drop nas (looking for motor response)

    if not os.path.exists(events_path):
        os.makedirs(events_path)
    
    fn = os.path.join(events_path, os.path.basename(onset).partition('_events')[0] + '_motor_events.tsv')
    
    print('file name & dir: ', fn)
    
    event_design_df.to_csv(fn, index=False)

860 02 01 860 02 01
Number of nonresponse trials:  5
file name & dir:  /Users/67981492/Desktop/loki_1/data/BOLD/sub-860/fmriprep/sub-860/ses-02/func/sub-0860_ses02_task-lokicat_run01_10202020_182758_motor_events.tsv
860 02 02 860 02 02
Number of nonresponse trials:  3
file name & dir:  /Users/67981492/Desktop/loki_1/data/BOLD/sub-860/fmriprep/sub-860/ses-02/func/sub-0860_ses02_task-lokicat_run02_10202020_183916_motor_events.tsv
860 02 03 860 02 03
Number of nonresponse trials:  3
file name & dir:  /Users/67981492/Desktop/loki_1/data/BOLD/sub-860/fmriprep/sub-860/ses-02/func/sub-0860_ses02_task-lokicat_run03_10202020_184937_motor_events.tsv
860 02 04 860 02 04
Number of nonresponse trials:  2
file name & dir:  /Users/67981492/Desktop/loki_1/data/BOLD/sub-860/fmriprep/sub-860/ses-02/func/sub-0860_ses02_task-lokicat_run04_10202020_185942_motor_events.tsv
860 02 05 860 02 05
Number of nonresponse trials:  2
file name & dir:  /Users/67981492/Desktop/loki_1/data/BOLD/sub-860/fmriprep/sub-860

860 09 05 860 09 05
Number of nonresponse trials:  0
file name & dir:  /Users/67981492/Desktop/loki_1/data/BOLD/sub-860/fmriprep/sub-860/ses-09/func/sub-0860_ses09_task-lokicat_run05_11182020_184404_motor_events.tsv
860 10 01 860 10 01
Number of nonresponse trials:  0
file name & dir:  /Users/67981492/Desktop/loki_1/data/BOLD/sub-860/fmriprep/sub-860/ses-10/func/sub-0860_ses10_task-lokicat_run01_11192020_175636_motor_events.tsv
860 10 02 860 10 02
Number of nonresponse trials:  0
file name & dir:  /Users/67981492/Desktop/loki_1/data/BOLD/sub-860/fmriprep/sub-860/ses-10/func/sub-0860_ses10_task-lokicat_run02_11192020_180656_motor_events.tsv
860 10 03 860 10 03
Number of nonresponse trials:  1
file name & dir:  /Users/67981492/Desktop/loki_1/data/BOLD/sub-860/fmriprep/sub-860/ses-10/func/sub-0860_ses10_task-lokicat_run03_11192020_181758_motor_events.tsv
860 10 04 860 10 04
Number of nonresponse trials:  0
file name & dir:  /Users/67981492/Desktop/loki_1/data/BOLD/sub-860/fmriprep/sub-860